In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from src.processing.splitting import time_based_split
from src.processing.scaling import scale_features
from src.processing.sequence_creator import create_sequences
from src.training.lstm_training import train_lstm
from src.training.hyperparameters_tuning import tune_hyperparameters

from config import config


df_btc = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "btc.csv",
    parse_dates=["date"],
    index_col="date",
)
df_eth = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "eth.csv",
    parse_dates=["date"],
    index_col="date",
)

## Split


In [9]:
X_train_btc, y_train_btc, X_val_btc, y_val_btc, X_test_btc, y_test_btc = (
    time_based_split(
        df_btc,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

X_train_eth, y_train_eth, X_val_eth, y_val_eth, X_test_eth, y_test_eth = (
    time_based_split(
        df_eth,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

In [10]:
print(y_test_btc.tail())
print(y_test_btc.head())

            logPriceChange
date                      
2022-12-15       -0.042190
2022-12-16        0.008828
2022-12-17       -0.002212
2022-12-18       -0.019176
2022-12-19        0.027989
            logPriceChange
date                      
2021-11-21       -0.042455
2021-11-22        0.022481
2021-11-23       -0.022639
2021-11-24        0.017512
2021-11-25       -0.066874


## Scale


In [11]:
X_train_scaled_btc, X_val_scaled_btc, X_test_scaled_btc, scaler_btc = scale_features(
    X_train_btc, X_val_btc, X_test_btc
)

X_train_scaled_eth, X_val_scaled_eth, X_test_scaled_eth, scaler_eth = scale_features(
    X_train_eth, X_val_eth, X_test_eth
)

## Sequence Creation


In [12]:
X_train_seq_btc, y_train_seq_btc = create_sequences(
    X_train_scaled_btc, y_train_btc, seq_length=30
)
X_val_seq_btc, y_val_seq_btc = create_sequences(
    X_val_scaled_btc, y_val_btc, seq_length=30
)
X_test_seq_btc, y_test_seq_btc = create_sequences(
    X_test_scaled_btc, y_test_btc, seq_length=30
)

print(f"X_train_seq_btc shape: {X_train_seq_btc.shape}")
print(f"y_train_seq_btc shape: {y_train_seq_btc.shape}")

X_train_seq_eth, y_train_seq_eth = create_sequences(
    X_train_scaled_eth, y_train_eth, seq_length=30
)
X_val_seq_eth, y_val_seq_eth = create_sequences(
    X_val_scaled_eth, y_val_eth, seq_length=30
)
X_test_seq_eth, y_test_seq_eth = create_sequences(
    X_test_scaled_eth, y_test_eth, seq_length=30
)

X_train_seq_btc shape: (1662, 30, 84)
y_train_seq_btc shape: (1662, 1)


# BTC


### Model Tuning

In [13]:
# study_lstm_btc = tune_hyperparameters(
#     X_train_seq_btc,
#     y_train_seq_btc,
#     X_val_seq_btc,
#     y_val_seq_btc,
#     train_lstm,
#     n_trials=50,
# )

In [21]:
len(y_test_seq_btc)

314

## Model Training

In [14]:
# best parameters from the Optuna study
# best_params_btc = study_lstm_btc.best_params
best_params_btc = {
    "num_lstm_layers": 3,
    "lstm_units_1": 192,
    "lstm_units_2": 128,
    "lstm_units_3": 32,
    "num_dense_layers": 2,
    "dense_units_1": 224,
    "dense_units_2": 192,
    "dropout_rate": 0.4,
    "learning_rate": 0.001277122777959738,
    "batch_size": 64,
}


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_btc[f"lstm_units_{i+1}"] for i in range(best_params_btc["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_btc[f"dense_units_{i+1}"]
        for i in range(best_params_btc["num_dense_layers"])
    ],
    "dropout_rate": best_params_btc["dropout_rate"],
    "learning_rate": best_params_btc["learning_rate"],
    "batch_size": best_params_btc["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_btc, history_btc = train_lstm(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    params=formatted_params,
)

Epoch 1/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.0083 - mae: 0.0691 - val_loss: 0.0016 - val_mae: 0.0328
Epoch 2/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0041 - mae: 0.0480 - val_loss: 0.0012 - val_mae: 0.0273
Epoch 3/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0024 - mae: 0.0353 - val_loss: 0.0012 - val_mae: 0.0268
Epoch 4/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0020 - mae: 0.0313 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 5/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - loss: 0.0021 - mae: 0.0317 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 6/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0020 - mae: 0.0312 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 7/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - loss: 0.0020 - mae: 0.0309 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 8/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 0.0020 - mae: 0.0306 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 9/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - lo

In [15]:
# save weights
model_btc.save_weights(
    config.DATA_DIR
    / "weights"
    / "multi_source_data"
    / "log_price_change_weights_btc.weights.h5"
)

#  generate predictions
y_pred_btc = model_btc.predict(X_test_seq_btc).flatten()

# create new df and save it to csv
predictions_btc = pd.DataFrame(
    {
        "date": y_test_btc.index[30:],
        "predictedLogPriceChange": y_pred_btc,
    }
)


predictions_btc.to_csv(
    config.DATA_DIR
    / "predictions"
    / "multi_source_data"
    / "log_price_change_predictions_btc.csv",
    index=False,
)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


# ETH

### Model Tuning

In [16]:
# study_lstm_eth = tune_hyperparameters(
#     X_train_seq_eth,
#     y_train_seq_eth,
#     X_val_seq_eth,
#     y_val_seq_eth,
#     train_lstm,
#     n_trials=50,
# )

## Model Training

In [18]:
# best parameters from the Optuna study
# best_params_eth = study_lstm_eth.best_params
best_params_eth = {
    "num_lstm_layers": 1,
    "lstm_units_1": 224,
    "num_dense_layers": 2,
    "dense_units_1": 96,
    "dense_units_2": 256,
    "dropout_rate": 0.2,
    "learning_rate": 0.0014383064200000338,
    "batch_size": 32,
}


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_eth[f"lstm_units_{i+1}"] for i in range(best_params_eth["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_eth[f"dense_units_{i+1}"]
        for i in range(best_params_eth["num_dense_layers"])
    ],
    "dropout_rate": best_params_eth["dropout_rate"],
    "learning_rate": best_params_eth["learning_rate"],
    "batch_size": best_params_eth["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_eth, history_eth = train_lstm(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    params=formatted_params,
)

Epoch 1/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0351 - mae: 0.1368 - val_loss: 0.0047 - val_mae: 0.0560
Epoch 2/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0087 - mae: 0.0679 - val_loss: 0.0030 - val_mae: 0.0423
Epoch 3/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0059 - mae: 0.0553 - val_loss: 0.0025 - val_mae: 0.0411
Epoch 4/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0043 - mae: 0.0470 - val_loss: 0.0016 - val_mae: 0.0313
Epoch 5/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0041 - mae: 0.0443 - val_loss: 0.0018 - val_mae: 0.0326
Epoch 6/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0042 - mae: 0.0437 - val_loss: 0.0015 - val_mae: 0.0299
Epoch 7/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0042 - mae: 0.0432 - val_loss: 0.0016 - val_mae: 0.0311
Epoch 8/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0041 - mae: 0.0433 - val_loss: 0.0015 - val_mae: 0.0307
Epoch 9/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - lo

In [19]:
# save weights
model_eth.save_weights(
    config.DATA_DIR
    / "weights"
    / "multi_source_data"
    / "log_price_change_weights_eth.weights.h5"
)

#  generate predictions
y_pred_eth = model_eth.predict(X_test_seq_eth).flatten()

# create new df and save it to csv
predictions_eth = pd.DataFrame(
    {
        "date": y_test_eth.index[30:],
        "predictedLogPriceChange": y_pred_eth,
    }
)


predictions_eth.to_csv(
    config.DATA_DIR
    / "predictions"
    / "multi_source_data"
    / "log_price_change_predictions_eth.csv",
    index=False,
)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
